<h1> Battle of Neighbourhoods in Munich
    
<h2> Business Problem


Munich is a beautyful city which has approximatly 1.5 Mio. inhabitants living in 25 neighbourhoods. In 2017 there were about 2700 restaurants, cafes, bars etc. spreaded over the city .

The following business case describes an imaginary team that want to acquire real estate in some neighbourhoods in the city of Munich and lease it to different tenants who plan to open a new restaurant.

They are not fixed to any type of cuisine so they first make a market analysis to look where are the best places to open a new restaurant. 
This analysis includes to look for which cuisine the competition is smallest in a specific neighbourhood. Another question that arises how many people are potential customers and what area a restaurant has to cover. 

With data from Foursquare the team wants to get an first overview, how many restaurants are in one neighbourhood to get an quick idea where the competition might be rough or weak or to check, in which neighbourhood a specific cusine might be best, dependend on venue data. 



In [1]:
import types
import pandas as pd
import numpy as np
import json # library to handle JSON files

# Libraries for importing data

import requests
from botocore.client import Config
import ibm_boto3

# Visualization libraries

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

import folium

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Machine Learning libraries

from sklearn.cluster import KMeans

print("Libraries installed and imported")

Libraries installed and imported


<h2> Data Section

Describes the data

##### Load additional data from csv and Excel

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
body_lat_lon = client_f77eb0a4a28b4c8a895ea7097b396fb9.get_object(Bucket='pythonbasicsfordatascienceproject-donotdelete-pr-ounhltxebnb12q',Key='neighbourhood_geodata.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body_lat_lon, "__iter__"): body_lat_lon.__iter__ = types.MethodType( __iter__, body_lat_lon )

#### Load geojson file for neighbourhoods

The geojson file is from open source air'b&b data base for munich

In [4]:
# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
munich_json = client_f77eb0a4a28b4c8a895ea7097b396fb9.get_object(Bucket='pythonbasicsfordatascienceproject-donotdelete-pr-ounhltxebnb12q', Key='neighbourhoods.geojson')['Body']
# add missing __iter__ method so pandas accepts body as file-like object
if not hasattr(munich_json, "__iter__"): munich_json.__iter__ = types.MethodType( __iter__, munich_json ) 


In [5]:
munich_data = json.load(munich_json)

### Create data frame with zip codes and attached coordinates

Postal Codes were collected muenchen.de. Munich has for every neighbourhood more than one zip code, so to make things easier only one zip code for every neighbourhood was chosen. The Latitude and Longitude were extracted from wikipedia.org and collected in one Excel sheet.

In [6]:
df_lat_lon = pd.read_excel(body_lat_lon)
df_lat_lon.set_index("Neighbourhood", drop = True, inplace = True)
df_lat_lon.head()

,Postal Code,Latitude,Longitude
Neighbourhood,,,
Altstadt-Lehel,80331,48.136111,11.572222
Ludwigsvorstadt-Isarvorstadt,80335,48.127222,11.564722
Maxvorstadt,80333,48.150000,11.569444
Schwabing-West,80809,48.161111,11.568889
Au-Haidhausen,81543,48.131944,11.588889


The data for Population, Area, Density and the percentage of whole population is from the open data portal of the city of Munich and was last updated on January 2019

In [7]:
df_pop = pd.read_csv(body_pop, encoding='ISO-8859-1')
df_pop.set_index("Neighbourhood", drop = True, inplace = True)
df_pop.head()

,Population,Area,Density,Percentage
Neighbourhood,,,,
Altstadt-Lehel,20422,314.57,65,1.39
Ludwigsvorstadt-Isarvorstadt,50620,440.14,115,3.46
Maxvorstadt,51642,429.79,120,3.53
Schwabing-West,65892,436.30,151,4.50
Au-Haidhausen,59752,421.96,142,4.08


In [8]:
df_munich = df_pop.join(df_lat_lon, how = 'outer')
df_munich

,Population,Area,Density,Percentage,Postal Code,Latitude,Longitude
Neighbourhood,,,,,,,
Altstadt-Lehel,20422,314.57,65,1.39,80331,48.136111,11.572222
Ludwigsvorstadt-Isarvorstadt,50620,440.14,115,3.46,80335,48.127222,11.564722
Maxvorstadt,51642,429.79,120,3.53,80333,48.150000,11.569444
Schwabing-West,65892,436.30,151,4.50,80809,48.161111,11.568889
Au-Haidhausen,59752,421.96,142,4.08,81543,48.131944,11.588889
Sendling,39953,393.87,101,2.73,80336,48.121389,11.541389
Sendling-Westpark,55405,781.45,71,3.78,81377,48.122222,11.531944
Schwanthalerhoehe,29663,207.02,143,2.02,80339,48.138900,11.541700
Neuhausen-Nymphenburg,95906,1291.45,74,6.55,80634,48.156944,11.516667


In [9]:
print("Population of Munich:", str(np.sum(df_munich.Population)), "inhabitants")
print("Average Population per Neighbourhood:", str(np.mean(df_munich.Population)), "inhabitants")
print("Number of Neighbourhoods:", str(len(df_munich.index)))

Population of Munich: 1464961 inhabitants
Average Population per Neighbourhood: 58598.44 inhabitants
Number of Neighbourhoods: 25


<h1> Foursquare data

<h3> Define Foursquare Credentails and Version

In [39]:
# The code was removed by Watson Studio for sharing.

#### Top 100 venues that are in Sendling within a radius of 500 meters.

In [11]:
neighborhood_latitude = df_munich.loc["Sendling", 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_munich.loc["Sendling", 'Longitude'] # neighborhood longitude value

neighborhood_name = "Sendling" # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of Sendling are 48.121389, 11.541389.


In [12]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2e73311db5810ffbf9f24b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Sendling',
  'headerFullLocation': 'Sendling, Munich',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 33,
  'suggestedBounds': {'ne': {'lat': 48.1258890045, 'lng': 11.54811744158758},
   'sw': {'lat': 48.1168889955, 'lng': 11.53466055841242}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51de8b18498e110bff0b1af1',
       'name': 'Hofladen am Stemmerhof',
       'location': {'address': 'Plinganserstr. 6',
        'crossStreet': 'Lindwurmstr.',
        'lat': 48.121631592087965,
        'lng': 11.540827499447134,
        'labeledLatLngs': [{'label': 'd

Transform JSON Data into dataframe

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
# To adress the neighbourhood pull them from index into a column

df_munich_reindexed = df_munich.reset_index(level = 0)
df_munich_reindexed.head()

,Neighbourhood,Population,Area,Density,Percentage,Postal Code,Latitude,Longitude
0,Altstadt-Lehel,20422,314.57,65,1.39,80331,48.136111,11.572222
1,Ludwigsvorstadt-Isarvorstadt,50620,440.14,115,3.46,80335,48.127222,11.564722
2,Maxvorstadt,51642,429.79,120,3.53,80333,48.150000,11.569444
3,Schwabing-West,65892,436.30,151,4.50,80809,48.161111,11.568889
4,Au-Haidhausen,59752,421.96,142,4.08,81543,48.131944,11.588889


In [16]:
munich_venues = getNearbyVenues(names=df_munich_reindexed["Neighbourhood"],
                                   latitudes=df_munich_reindexed["Latitude"],
                                   longitudes=df_munich_reindexed["Longitude"]
                                  )

Altstadt-Lehel
Ludwigsvorstadt-Isarvorstadt
Maxvorstadt
Schwabing-West
Au-Haidhausen
Sendling
Sendling-Westpark
Schwanthalerhoehe
Neuhausen-Nymphenburg
Moosach
Milbertshofen-Am Hart
Schwabing-Freimann
Bogenhausen
Berg am Laim
Trudering-Riem
Ramersdorf-Perlach
Obergiesing
Untergiesing-Harlaching
Thalkirchen-Obersendling-Forstenried-Fuerstenried-Solln
Hadern
Pasing-Obermenzing
Aubing-Lochhausen-Langwied
Allach-Untermenzing
Feldmoching-Hasenbergl
Laim


In [17]:
print(munich_venues.shape)
munich_venues.head()

(686, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt-Lehel,48.136111,11.572222,Asamkirche (St. Johann Nepomuk),48.135053,11.569746,Church
1,Altstadt-Lehel,48.136111,11.572222,Kustermann,48.136242,11.574897,Department Store
2,Altstadt-Lehel,48.136111,11.572222,Galeria Gourmet,48.137432,11.573217,Gourmet Shop
3,Altstadt-Lehel,48.136111,11.572222,Hirmer,48.138023,11.572046,Men's Store
4,Altstadt-Lehel,48.136111,11.572222,La Burrita,48.136143,11.574489,Burrito Place


In [18]:
munich_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Allach-Untermenzing,4,4,4,4,4,4
Altstadt-Lehel,100,100,100,100,100,100
Au-Haidhausen,60,60,60,60,60,60
Aubing-Lochhausen-Langwied,6,6,6,6,6,6
Berg am Laim,4,4,4,4,4,4
Bogenhausen,37,37,37,37,37,37
Feldmoching-Hasenbergl,16,16,16,16,16,16
Hadern,5,5,5,5,5,5
Laim,30,30,30,30,30,30


In [19]:
print('There are {} uniques categories.'.format(len(munich_venues['Venue Category'].unique())))

There are 163 uniques categories.


In [20]:
# one hot encoding
munich_onehot = pd.get_dummies(munich_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
munich_onehot['Neighbourhood'] = munich_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [munich_onehot.columns[-1]] + list(munich_onehot.columns[:-1])
munich_onehot = munich_onehot[fixed_columns]

munich_onehot.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,Bagel Shop,Bakery,...,Thai Restaurant,Theme Restaurant,Toy / Game Store,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Altstadt-Lehel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
munich_grouped = munich_onehot.groupby('Neighbourhood').mean().reset_index()
munich_grouped

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,Bagel Shop,Bakery,...,Thai Restaurant,Theme Restaurant,Toy / Game Store,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Allach-Untermenzing,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.00
1,Altstadt-Lehel,0.010000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.00,0.01
2,Au-Haidhausen,0.033333,0.000000,0.000000,0.000000,0.00,0.033333,0.000000,0.000000,0.016667,...,0.016667,0.000000,0.000000,0.000000,0.016667,0.000000,0.016667,0.016667,0.00,0.00
3,Aubing-Lochhausen-Langwied,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.166667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
4,Berg am Laim,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
5,Bogenhausen,0.000000,0.000000,0.000000,0.000000,0.00,0.054054,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
6,Feldmoching-Hasenbergl,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.125000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
7,Hadern,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
8,Laim,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.066667,0.033333,0.000000,0.000000,0.000000,0.00,0.00
9,Ludwigsvorstadt-Isarvorstadt,0.000000,0.018868,0.018868,0.000000,0.00,0.037736,0.018868,0.000000,0.018868,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075472,0.00,0.00


In [22]:
num_top_venues = 5

for hood in munich_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = munich_grouped[munich_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allach-Untermenzing----
                 venue  freq
0                Hotel  0.25
1            Racetrack  0.25
2    Trattoria/Osteria  0.25
3  Bavarian Restaurant  0.25
4    Afghan Restaurant  0.00


----Altstadt-Lehel----
                 venue  freq
0    German Restaurant  0.07
1                 Café  0.06
2       Clothing Store  0.05
3  Bavarian Restaurant  0.04
4          Coffee Shop  0.04


----Au-Haidhausen----
               venue  freq
0               Café  0.05
1  Indian Restaurant  0.05
2  Afghan Restaurant  0.03
3  French Restaurant  0.03
4       Concert Hall  0.03


----Aubing-Lochhausen-Langwied----
          venue  freq
0         Hotel  0.17
1  Soccer Field  0.17
2   Supermarket  0.17
3      Bus Stop  0.17
4        Bakery  0.17


----Berg am Laim----
               venue  freq
0      Metro Station  0.25
1        Supermarket  0.25
2           Bus Stop  0.25
3  German Restaurant  0.25
4  Afghan Restaurant  0.00


----Bogenhausen----
                  venue  freq
0    It

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = munich_grouped['Neighbourhood']

for ind in np.arange(munich_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(munich_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allach-Untermenzing,Hotel,Trattoria/Osteria,Racetrack,Bavarian Restaurant,Wine Shop,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,Altstadt-Lehel,German Restaurant,Café,Clothing Store,Coffee Shop,Bavarian Restaurant,Museum,Hotel,Church,Italian Restaurant,Cosmetics Shop
2,Au-Haidhausen,Indian Restaurant,Café,Restaurant,Beach,Asian Restaurant,Beer Garden,Afghan Restaurant,Ice Cream Shop,French Restaurant,Concert Hall
3,Aubing-Lochhausen-Langwied,German Restaurant,Hotel,Supermarket,Bakery,Bus Stop,Soccer Field,Doner Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant
4,Berg am Laim,Metro Station,Supermarket,German Restaurant,Bus Stop,Doner Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Drugstore


<h1> Cluster Neighbourhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [25]:
# set number of clusters
kclusters = 5

munich_grouped_clustering = munich_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(munich_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 2, 4, 1, 2, 0, 1, 1], dtype=int32)

In [26]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [27]:
munich_merged = df_munich_reindexed

munich_merged = munich_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

munich_merged.head() # check the last columns!

,Neighbourhood,Population,Area,Density,Percentage,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Altstadt-Lehel,20422,314.57,65,1.39,80331,48.136111,11.572222,1,German Restaurant,Café,Clothing Store,Coffee Shop,Bavarian Restaurant,Museum,Hotel,Church,Italian Restaurant,Cosmetics Shop
1,Ludwigsvorstadt-Isarvorstadt,50620,440.14,115,3.46,80335,48.127222,11.564722,1,Café,Vietnamese Restaurant,Italian Restaurant,Bar,Ice Cream Shop,Plaza,Asian Restaurant,Pub,Japanese Restaurant,Cocktail Bar
2,Maxvorstadt,51642,429.79,120,3.53,80333,48.150000,11.569444,1,Café,Vietnamese Restaurant,Italian Restaurant,Art Museum,Bakery,Drugstore,Pizza Place,Japanese Restaurant,History Museum,Doner Restaurant
3,Schwabing-West,65892,436.30,151,4.50,80809,48.161111,11.568889,1,Vietnamese Restaurant,Italian Restaurant,Indian Restaurant,Greek Restaurant,Café,Bar,Bakery,Sushi Restaurant,Restaurant,Mexican Restaurant
4,Au-Haidhausen,59752,421.96,142,4.08,81543,48.131944,11.588889,1,Indian Restaurant,Café,Restaurant,Beach,Asian Restaurant,Beer Garden,Afghan Restaurant,Ice Cream Shop,French Restaurant,Concert Hall


<h2> Examine Clusters

<h3> Cluster 1

In [31]:
munich_merged.loc[munich_merged['Cluster Labels'] == 0, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,Population,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,90790,81477,48.089000,11.519000,0,Bus Stop,Trattoria/Osteria,Supermarket,Italian Restaurant,Tennis Court,Wine Shop,Diner,Event Space,Ethiopian Restaurant,Electronics Store
19,48945,80689,48.110556,11.465278,0,Bus Stop,Cafeteria,Bistro,Salon / Barbershop,Wine Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Drugstore


<h3> Cluster 2

In [32]:
munich_merged.loc[munich_merged['Cluster Labels'] == 1, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,Population,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,20422,80331,48.136111,11.572222,1,German Restaurant,Café,Clothing Store,Coffee Shop,Bavarian Restaurant,Museum,Hotel,Church,Italian Restaurant,Cosmetics Shop
1,50620,80335,48.127222,11.564722,1,Café,Vietnamese Restaurant,Italian Restaurant,Bar,Ice Cream Shop,Plaza,Asian Restaurant,Pub,Japanese Restaurant,Cocktail Bar
2,51642,80333,48.150000,11.569444,1,Café,Vietnamese Restaurant,Italian Restaurant,Art Museum,Bakery,Drugstore,Pizza Place,Japanese Restaurant,History Museum,Doner Restaurant
3,65892,80809,48.161111,11.568889,1,Vietnamese Restaurant,Italian Restaurant,Indian Restaurant,Greek Restaurant,Café,Bar,Bakery,Sushi Restaurant,Restaurant,Mexican Restaurant
4,59752,81543,48.131944,11.588889,1,Indian Restaurant,Café,Restaurant,Beach,Asian Restaurant,Beer Garden,Afghan Restaurant,Ice Cream Shop,French Restaurant,Concert Hall
5,39953,80336,48.121389,11.541389,1,Café,Vietnamese Restaurant,Supermarket,Gym,German Restaurant,Gastropub,Spanish Restaurant,Doner Restaurant,Soup Place,Bistro
7,29663,80339,48.138900,11.541700,1,Hotel,Café,Bar,Pizza Place,Vietnamese Restaurant,French Restaurant,Trattoria/Osteria,German Restaurant,Doner Restaurant,Grocery Store
11,69676,80939,48.180556,11.602778,1,Metro Station,Cafeteria,Gym / Fitness Center,Turkish Restaurant,Beer Garden,Bank,Café,Farmers Market,Falafel Restaurant,Event Space
12,82138,81675,48.148056,11.616667,1,Italian Restaurant,Gym / Fitness Center,Supermarket,Hotel,Café,Bus Stop,Drugstore,Asian Restaurant,Farmers Market,Movie Theater
16,51499,81547,48.111111,11.594444,1,Supermarket,Plaza,Hookah Bar,Grocery Store,Burger Joint,Greek Restaurant,Ice Cream Shop,Convenience Store,Drugstore,Shipping Store


<h3> Cluster 3

In [33]:
munich_merged.loc[munich_merged['Cluster Labels'] == 2, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,Population,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,55405,81377,48.122222,11.531944,2,Supermarket,Restaurant,Café,Playground,Turkish Restaurant,Hotel,Indian Restaurant,Bakery,Ethiopian Restaurant,Electronics Store
8,95906,80634,48.156944,11.516667,2,Tram Station,Plaza,Tennis Court,Canal,Supermarket,Italian Restaurant,Bank,Bakery,Light Rail Station,Ice Cream Shop
9,51537,80997,48.183333,11.516667,2,Bakery,Plaza,Supermarket,Bank,Hotel,German Restaurant,Italian Restaurant,Light Rail Station,Food,Drugstore
14,67009,81735,48.116667,11.658333,2,Pub,Restaurant,Drugstore,Supermarket,Bakery,Wine Shop,Diner,Event Space,Ethiopian Restaurant,Electronics Store
15,108244,81739,48.102800,11.625000,2,Supermarket,German Restaurant,Plaza,Hotel,Bus Stop,Dessert Shop,Bakery,Italian Restaurant,Organic Grocery,Diner
21,42305,81243,48.158333,11.419444,2,German Restaurant,Hotel,Supermarket,Bakery,Bus Stop,Soccer Field,Doner Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant
22,30737,80995,48.200000,11.452778,2,Hotel,Trattoria/Osteria,Racetrack,Bavarian Restaurant,Wine Shop,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
23,59391,80933,48.211100,11.541700,2,Supermarket,Ice Cream Shop,Bakery,Bus Stop,Plaza,Bus Line,Club House,Lottery Retailer,Light Rail Station,Italian Restaurant


<h3> Cluster 4

In [34]:
munich_merged.loc[munich_merged['Cluster Labels'] == 3, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,Population,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,51937,81545,48.092222,11.560556,3,Café,Tram Station,German Restaurant,Plaza,Cultural Center,Cupcake Shop,Event Space,Ethiopian Restaurant,Electronics Store,Drugstore


<h3> Cluster 5

In [35]:
munich_merged.loc[munich_merged['Cluster Labels'] == 4, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,Population,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,73617,80937,48.198333,11.576389,4,Metro Station,Lounge,Bus Stop,German Restaurant,Mexican Restaurant,Donut Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
13,43068,81671,48.122222,11.627778,4,Metro Station,Supermarket,German Restaurant,Bus Stop,Doner Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Drugstore


<h3> Visualize results with Folium

Choropleth map to visualize the population in each neighbourhood

In [37]:
address = 'Munich, BY'

geolocator = Nominatim(user_agent="mu_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Munich are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Munich are 48.1371079, 11.5753822.


In [38]:
# create a choropleth map of munich showing neighbourhoods with population

munich_venue_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

munich_venue_clusters.choropleth(
    geo_data=munich_data,
    data=df_munich_reindexed,
    columns=['Neighbourhood', 'Density'],
    key_on='feature.properties.neighbourhood',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Population in Munich',
    reset=True
)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(munich_merged['Latitude'], munich_merged['Longitude'], munich_merged['Neighbourhood'], munich_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(munich_venue_clusters)
       
munich_venue_clusters